# test code for cancer KZFP figure: figure 4 for KZFP review paper:
- load softwares needed
- 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from bisect import bisect_left

Matplotlib is building the font cache; this may take a moment.


In [3]:
print("why")

why
